# Solving-Hamilton-Jacobi-Bellman Equations (via FBSDEs)
#### Frederik Kelbel, Imperial College London

In [1]:
import torch
import plotly.graph_objects as go
import numpy as np
from operators import div, Δ, D, mdotb, bdotm, mdotm, bdotb
from FBSDEs import FBSDESolver
from pdes import HBJ
from plotly.subplots import make_subplots
from configs import CONFIG_FBSDES as MODEL_CONFIG
from itertools import product

### Linear-quadratic control problem 1-dimensional (Riccati Equation) 

Let $(\Omega, \mathcal{F}, \{\mathcal{F}_t\}_{t\in [0, T]}, \mathbb{P})$. We consider
$$
\begin{cases}
dX_s = [H_sX_s + M_s u_s] ds + \sigma_s dW_s, \; s \in [0, T] \\
X_0 = x > 0
\end{cases},
$$

We aim to maximise
$$
J^u(t, x) := \mathbb{E}^{t, x} \Big[ \int_t^T X_s^T C_s X_s + \frac{1}{2}u_s^T D_s u_s ds + X_T^T R X_T\Big],
$$
with $C(t) = C \leq 0, R \leq 0$, and $D=D(t) < -\delta < 0$ given and deterministic ($\delta > 0$ some constant).

We write down the problem in its primal form as
$$
\begin{cases}
\partial_t J(t, x) + \inf_{u} \Big\{ \frac{1}{2} \sigma^2 \partial_{xx} J(t, x) + [H x + M u] \partial_x J(t, x) + C x^2 + \frac{1}{2}D u^2 \Big\} = 0 \text{ on $[0, T] \times (-\infty, \infty)$}
\\
J(T, x) = Rx^2 \text{ $\forall x \in \mathbb{R}$}
\end{cases}
$$

$$
\begin{cases}
        d\tilde{X}_t = (H(t, \tilde{X}_t) + \sigma(t, \tilde{X}_t) \Gamma^T \tilde{U}_t)dt + \sigma(t, \tilde{X}_t) dW_t, \quad t \in [0, T] \\
        \tilde{X}_0 = x
\end{cases}
$$
$$
\begin{cases}
        d\tilde{Y}_t = C(\tilde{X}_t) dt + \frac{1}{2}(\tilde{Z}_t^T \Gamma D^{-1} \Gamma^T \tilde{Z}_t)(t, \tilde{X}_t) dt + \tilde{Z}_t^T  dW_t, \quad t \in [0, T] \\
        \tilde{Y}_T = g(\tilde{X}_T)
\end{cases}.
$$

In [2]:
class LQR(HBJ):
    def __init__(self):
        super().__init__()
        self.H = lambda X, t: 0.3
        M = 2.0
        self.C = lambda X: 2.0
        R = 0.001
        self.D = 0.2
        self.sigma = lambda X, t: 0.3
        
        self.Gamma = lambda X, t: M/self.σ
        self.terminal_condition = lambda X: R*X**2
        
        self.var_dim = 1
        self.terminal_time = 1      

In [3]:
LQR_MODEL_CONFIG = {
    "batch_size": 64,
    "num_discretisation_steps": 10,
    "hidden_dim": 64,
    "learning_rate": 5e-3,
    "loss_weights": (1, 1),
    "lr_decay": 0.99,
    "network_type": "GRU",
    "optimiser": "Adam"
}
eq = LQR()
model = LQR_MODEL_CONFIG
solver = FBSDESolver(model, eq)
loss = np.array(list(solver.train(0.5, 3)))
print(loss)

TypeError: expected Tensor as element 0 in argument 1, but got float